In [ ]:
# Google Text-to-Speech
!pip install gTTS

# Pygame (for audio playback)
!pip install pygame

# SpeechRecognition (for speech-to-text functionality)
!pip install SpeechRecognition

# LangChain and related community libraries
!pip install langchain langchain-community langchain-openai

# FAISS (for vector store)
!pip install faiss-cpu  # or faiss-gpu if you have a GPU

# OpenAI (for OpenAI API access)
!pip install openai

!pip install faiss-cpu

!pip install pyaudio


In [ ]:
import io

from gtts import gTTS

import pygame

import speech_recognition as sr

from langchain_community.document_loaders import TextLoader

from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts.chat import ChatPromptTemplate

from langchain_openai import ChatOpenAI

from langchain_community.callbacks.manager import get_openai_callback

from langchain_core.chat_history import BaseChatMessageHistory

from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableParallel
   
from langchain_community.retrievers.llama_index import LlamaIndexRetriever

from langchain_core.chat_history import InMemoryChatMessageHistory

from langchain_core.output_parsers.string import StrOutputParser

from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_core.runnables.utils import ConfigurableFieldSpec

from openai import OpenAI


In [2]:
#speech o text

# Initialize recognizer
r = sr.Recognizer()

def get_voice_input():
    # Use the microphone as source
    with sr.Microphone() as source:
        print("Listening...")
        # Adjust for ambient noise to improve accuracy
        r.adjust_for_ambient_noise(source)
        
        # Listen to the input
        audio = r.listen(source,timeout=10)
        
        try:
            # Use Google's speech recognition to transcribe the audio
            text = r.recognize_google(audio)
            print(f"You said: {text}")
            #print("i am here")
            return text
        except sr.UnknownValueError:
            print("Could not understand the audio")
            #speak_text("Could you please repeat it again")
            #print("i struck")
            return "customer order is not clear ask him to repeat it"
        except sr.RequestError as e:
            print(f"Error with the recognition service: {e}")
            return None

# Call the function to get voice input



In [3]:
#text to speech

def speak_text(text):
    #Initialize Pygame mixer
    pygame.mixer.init()
    
    # The text you want to convert to speech
    text = text
    
    # Language in which you want to convert
    language = 'en'
    
    # Passing the text and language to the engine
    speech = gTTS(text=text, lang=language, slow=False)
    
    # Use an in-memory bytes buffer
    mp3_fp = io.BytesIO()
    speech.write_to_fp(mp3_fp)
    mp3_fp.seek(0)  # Reset buffer position to the beginning
    
    # Load the speech from the buffer
    pygame.mixer.music.load(mp3_fp, 'mp3')
    
    # Play the speech
    pygame.mixer.music.play()
    
    # Keep the script running until the speech finishes playing
    while pygame.mixer.music.get_busy():
        continue



In [4]:
import os


os.environ['OPENAI_API_KEY'] =  'Enter openai_api key here'
client = OpenAI()

In [5]:

#embedding and retriver

loader = TextLoader("src/Document/Menu.txt")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(texts, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [6]:
docs = retriever.invoke("I want one queen bee")

In [ ]:
docs

In [8]:

message = [
    # System prompt defining the assistant's role and guidelines
    ("system",("""
You are a friendly and professional waiter at a restaurant called Boba Bee.
Responsibilities include:
    - Assisting customers by accurately taking their orders.
    - Addressing special requests or customizations.
    - Ask for customization for every item.
    - Continue until the customer has finished ordering.
        Example that the customer finished ordering: Listen for cues like "That's all" or "I'm done."
    - Confirm the order with the customer.
        Examples for confirmation: "You ordered 1 Bubble Tea with extra tapioca pearls and 1 Honey Lemon Tea with no sugar. May I confirm your order?",
                                   "So far, I have 1 Green Tea with honey and 1 Iced Coffee with oat milk. Shall I proceed with this order?"
    - After confirmation, should summarize the order, which must include the word "Thank you."
        Examples for summarization and conclusion: "Here’s your order summary: 1 Bubble Tea with extra tapioca pearls and 1 Honey Lemon Tea with no sugar. Thank you!",
                                                   "Your order summary: 1 Matcha Latte with almond milk, 1 Mango Smoothie. Thank you for your order!"

Guidelines:
    - Keep replies concise.
    - Do not provide item or customization descriptions unless explicitly asked.
    - Do not ask for confirmation or summarize what the customer orders until the customer indicates they have finished ordering. Instead, ask if the customer wants to add more items.
    - Do not skip asking for confirmation of the order with the customer.
    - Only provide the items and customizations present in the menu list provided in the context.
    - If an item is mispronounced but present in the menu, add it silently unless you do not understand it.
    - Do not repeat the customer's selections and customizations after every selection unless you need to correct the spelling. Ask questions instead.

"The available menu items are provided below in delimiters:
    - ""Menu:{context}"".
    """)),

    # Start taking orders
    ("human", "{question}"),
    ("human", "can I get {question}"),  # Customer's first order
    ("ai", "Would you like to customize it?"),  # Asking for customization
    ("human", "{question}"),  # Customer's customization (if any)

    # Ask if they want to add more items
    ("ai", "Would you like to add more items to your order?"),
    ("human", "{question}"),  # Customer's response (Yes/No)

    # If customer says yes, take next item
    ("ai", "Please tell me the next item you'd like to order."),
    ("human", "{question}"),  # Customer's next order
    ("ai", "Would you like to customize it?"),
    ("human", "{question}"),  # Customer's customization (if any)

    # Ask again if they want to add more items
    ("ai", "Would you like to add more items to your order?"),
    ("human", "{question}"),  # Customer's response (Yes/No)

    # Proceed to confirmation
    ("ai", "You ordered:\n May I confirm your order before proceeding?"),
    ("human", "{question}"),  # Customer's confirmation (e.g., "Yes, please.")

    # Summarize the order
    ("ai", "Here is your order summary:\n Thank you")
]

prompt = ChatPromptTemplate.from_messages(message)


In [9]:
store={}
def get__by_session_id(session_id: str,store) -> BaseChatMessageHistory:
    if session_id not in store:
        #print(session_id)
        #print(store)
        store[session_id] = InMemoryChatMessageHistory()
        #print(store[session_id])
    return store[session_id]    

In [10]:
#model

model = ChatOpenAI(model="gpt-4o", temperature=0.2)
def get_assistant_response(user_input,order_number,store):
    
    
    chain = (
        RunnableParallel({"context": RunnableLambda(lambda x: x["question"]) | retriever, "question": RunnablePassthrough()})
        | prompt
        | model
        | StrOutputParser()
    )
    config={"configurable": {"session_id": order_number,"store":store}}
    chain_with_history = RunnableWithMessageHistory(
    chain,
    get__by_session_id,
    input_messages_key="question",
    history_messages_key="order_number",
    history_factory_config = [ConfigurableFieldSpec(id="session_id", annotation=str),
                              ConfigurableFieldSpec(id="store", annotation=dict)]
    
        
)
    #print(chain)
    with get_openai_callback() as cb:
    # Invoke the chain with history
        response=chain_with_history.invoke({"question":user_input},config=config)
        # Print token usage
        print(f"Prompt tokens used: {cb.prompt_tokens}")
        print(f"Completion tokens used: {cb.completion_tokens}")
        print(f"Total tokens used: {cb.total_tokens}")
        print(f"Total cost used: {cb.total_cost}")

    return response
    

In [15]:
def main(order_number):
    order_summary=None
    store={}
    assistant_response=' '
    speak_text("Hi welcome to Bobba Bee how can I help you today")
    while "thank you" not in assistant_response.lower():
        #print("i am in the loop")
        customer_order =get_voice_input()  
        if customer_order:
            print(f"Final recognized text: {customer_order}")
            assistant_response = get_assistant_response(customer_order,order_number,store)
            #speak_text(assistant_response)
            print(assistant_response)
    if "thank you" in assistant_response.lower():
        print("order number:",order_number, end='\n')
        print(assistant_response)
        #print(store)
        store={}

    main(order_number+1)

In [ ]:
%%time
if __name__ == "__main__":
    
    
    main(order_number=1)